# CEE6501 — Coding Assignment, Week 7

**Assigned:** 02/27/2026 (Week 7)  
**Due:** 03/12/2026

**Canvas Submission Link:**  <https://gatech.instructure.com/courses/517856/assignments/2320764>

---

## Logistics

### 💻 Assignment Format

This is a **coding assignment**.

- Complete the assignment by **executing and completing all tasks in the notebook cells below**
- The notebook should be run and completed in **Google Colab**
- Your submission **must be a link to a functioning Google Colab notebook**

You may use any local tools (VS Code, JupyterLab, etc.) while working, but the **final submitted work must run correctly in Colab**.


### 📤 Submission Instructions

- Submit **one link** to your Google Colab notebook on Canvas
- Ensure that:
  - All cells run **top-to-bottom without errors**
  - All required outputs are visible
  - The notebook reflects your final answers

### ✅ Checklist Before Submitting

- [ ] All notebook cells completed
- [ ] Code runs without errors from a fresh runtime
- [ ] Outputs and plots are clearly visible
- [ ] Colab link opens and runs correctly
- [ ] Correct notebook submitted on Canvas

### 🤝 Collaboration / AI tools
You may discuss concepts with classmates and you may use AI tools to help you learn,
but **your submitted code must be written by you and you must understand it**.
If you used outside help, add a short note in the final reflection cell.

---

## --- Google Colab environment setup ---

The cell below only needs to run when the notebook is opened in Google Colab.

This code will not affect code execution locally in VS-code + conda environment.

Google Colab starts each session with its own **preloaded versions** of common Python (currently 3.12.12) and Python packages (NumPy, SciPy, etc.).  
If we install different package versions once loaded, Python cannot switch to them while it is already running.

### What will happen
When you run the setup cell below in Google Colab:

1. The required package versions are installed
2. The runtime is **automatically restarted** so the new versions can be loaded  
3. You may see the message **“Your session crashed for an unknown reason.”**  
   → This is expected and normal

After the restart, rerun the notebook and check the **version check cell** to confirm package versions are correct.

### Runtime menu notes
- **Runtime → Restart session**  
  Restarts Python but keeps installed packages and saved files

- **Runtime → Disconnect and delete runtime**  
  Resets Colab completely to its default environment (packages will need to be reinstalled)

In [26]:
# ============================================================
# Google Colab environment setup (pinned versions)
# ============================================================

import sys
import os
import subprocess

if "google.colab" in sys.modules:
    print("Running in Google Colab")
    print("Python version:", sys.version.split()[0])

    # ---- Required package versions --------------------------
    requirements = {
        "numpy": "2.4.0",
        "scipy": "1.16.3",
        "matplotlib": "3.10.8",
        "pandas": "2.3.3",
        "plotly": "6.5.2"
    }

    # ---- Check currently loaded versions --------------------
    restart_needed = False

    for pkg, required_version in requirements.items():
        try:
            module = __import__(pkg)
            installed_version = module.__version__
        except Exception:
            installed_version = None

        print(f"{pkg}: {installed_version} (required: {required_version})")

        if installed_version != required_version:
            restart_needed = True

    # ---- Install if needed ----------------------------------
    if restart_needed:
        print("\nInstalling pinned package versions...")

        pip_args = [
            f"{pkg}=={ver}" for pkg, ver in requirements.items()
        ]

        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", "-q", *pip_args]
        )

        print("Installation complete.")
        print("Restarting runtime to load correct packages...")

        # This will appear as a "crash" in Colab — expected behavior
        os.kill(os.getpid(), 9)

    else:
        print("\nAll required package versions already installed.")

else:
    print("Not running in Google Colab — setup skipped.")
    print("Python version:", sys.version.split()[0])

Not running in Google Colab — setup skipped.
Python version: 3.12.12


In [27]:
# --- Version check ---
import numpy
import scipy
import matplotlib
import pandas
import plotly

print("numpy:", numpy.__version__)
print("scipy:", scipy.__version__)
print("matplotlib:", matplotlib.__version__)
print("pandas:", pandas.__version__)
print("plotly:", plotly.__version__)

numpy: 2.4.0
scipy: 1.16.3
matplotlib: 3.10.8
pandas: 2.3.3
plotly: 6.5.2


---
---

## Imports

Run this cell once before starting the assignment.

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.graph_objects as go

np.set_printoptions(precision=3, suppress=True)


---

## Assignment: Plotting Deformed Shape (Beams & Frames)

In the handwritten part, you solved:

- One simple **beam system**
- One simple **frame system**

In this coding assignment, you will write two plotting functions to visualize the **undeformed** and **deformed** shapes:

- `plot_deformed_beam_2d(...)`
- `plot_deformed_frame_2d(...)`

Use your hand-solved displacement results to drive the plots, and **verify the deformed shapes are consistent with your manual solutions**.

At minimum, each plotting function must show:

- the **undeformed** structure
- the **deformed** structure (with an appropriate scale factor)

You are encouraged to add extra visualization features, for example:

- node markers and node numbers  
- element numbers  
- support / boundary condition markers  
- load arrows (if available)  
- a legend and a title including the deformation scale  
- equal axis scaling so geometry is not distorted  

The goal is to generate a plot that is **correct**, **looks good**, and is **easy to interpret**.

## Generic Input Format

Your plotting functions must accept:

- `nodes`: dictionary of node coordinates
- `elements`: dictionary of element connectivity (at minimum `(i, j)`)
- `U`: global displacement vector
- `scale`: linear scaling factor for deformations
- `npts`: number of interpolation points for local element deformation

You may store additional properties (like $A$, and $E$) in `elements`, but plotting only requires connectivity.



---

## Problem 1 — Beam System (Bending Only)

Consider a 2D beam element of length $L$ with local coordinate $x \in [0, L]$ measured from node $i$.

Each node has two degrees of freedom:

- $u_y$ : transverse translation
- $\theta$ : rotation

So the global displacement vector has length $2n$

### DOF Mapping (1-Based Node Numbering)

For node $i$ with 1-based node numbering ($i = 1, 2, \dots, n$), the global displacement vector is ordered as

$$
U =
\begin{bmatrix}
u_{y1} \\
\theta_1 \\
u_{y2} \\
\theta_2 \\
\vdots \\
u_{yn} \\
\theta_n
\end{bmatrix}
$$

For node $i$ with 1-based node numbering ($i = 1, 2, \dots, n$),

$$
u_{yi} = U_{2i-1},
\qquad
\theta_i = U_{2i}
$$

If nodes are numbered starting from 0 ($i = 0, 1, \dots, n-1$),

$$
u_{yi} = U_{2i},
\qquad
\theta_i = U_{2i+1}
$$


### Beam Shape Functions (Euler–Bernoulli)

Consider a beam element of length $L$ with local coordinate $x \in [0, L]$ starting from node $i$ to node $j$.

Each element has the nodal degrees of freedom

$$
[\, u_{yi}, \theta_i, u_{yj}, \theta_j \,]
$$

The transverse displacement field within the element is interpolated as

$$
u_y(x) = N_1(x)u_{yi} + N_2(x)\theta_i + N_3(x)u_{yj} + N_4(x)\theta_j
$$

where the cubic shape functions are written directly in terms of $x$:

$$
\begin{aligned}
N_1(x) &= 1 - 3\frac{x^2}{L^2} + 2\frac{x^3}{L^3} \\[6pt]
N_2(x) &= x - 2\frac{x^2}{L} + \frac{x^3}{L^2} \\[6pt]
N_3(x) &= 3\frac{x^2}{L^2} - 2\frac{x^3}{L^3} \\[6pt]
N_4(x) &= -\frac{x^2}{L} + \frac{x^3}{L^2}
\end{aligned}
$$

These shape functions ensure:

- Displacement continuity at the nodes  
- Rotation continuity at the nodes  
- A smooth cubic deflection curve  

### Important

Do not draw the deformed frame using only straight lines between displaced nodes

Evaluate and $u_y(x)$ at multiple points along each element to obtain a smooth deformed shape

Connected elements must share the same nodal DOFs from the global vector $U$, so joints remain connected and rotations are compatible

### Check Against Your Manual Frame Solution

- Does the frame sway or bend in the correct direction
- Do joints remain connected
- Does the deformed shape look physically reasonable

### Conceptual Steps — Beam Deformation Plot

1. **Loop over elements**  
   Get node coordinates and compute the element length $L$

2. **Extract global nodal DOFs**  
   From the global vector $U$, extract  
   $[\,u_{yi}, \theta_i, u_{yj}, \theta_j\,]$

3. **Interpolate along the element**  
   For $x \in [0, L]$, compute the transverse displacement using cubic beam shape functions  
   $$
   u_y(x) = N_1(x)u_{yi} + N_2(x)\theta_i + N_3(x)u_{yj} + N_4(x)\theta_j
   $$

4. **Build deformed coordinates**  
   Add the scaled transverse displacement directly to the global geometry

5. **Plot the result**  
   Draw the undeformed straight element and the smooth cubic deformed curve

Because the beam lies along the global axis, the local and global coordinate systems are identical, so no transformation is required.

In [29]:
def plot_deformed_beam_2d(nodes, elements, U, scale=1.0, npts=50):
    """
    Problem 1 — Beam

    Plot undeformed and deformed shape for a 2D beam (bending only).

    Inputs:
        nodes    : {node_id: (x, y)}
        elements : {elem_id: (i, j)}
        U        : global displacement vector [v, th] per node
        scale    : deformation scale factor (for visualization only)
        npts     : number of interpolation points per element

    Steps you must implement:
        1) Loop through elements
        2) Compute element length L and direction cosines
        3) Extract (v_i, th_i, v_j, th_j) from U
        4) Evaluate v(x) using cubic shape functions
        5) Build global deformed coordinates and plot
    """

    # TODO: implement


In [30]:
# Write any additional helper function(s) that you want to call from your plotting function

In [31]:
# Define nodes, elements, and U for your beam example

nodes = {}
elements = {}
U = []

# Then call plot_deformed_beam_2d(...)

---

# Problem 2 — Frame System (Axial + Bending)

Consider a 2D frame element of length $L$ with local coordinate $x \in [0, L]$ measured from node $i$.

Each node has three degrees of freedom:

- $u_x$ : axial translation
- $u_y$ : transverse translation
- $\theta$ : rotation

So the global displacement vector has length $3n$



## DOF Mapping

For node $i$ with 1-based node numbering ($i = 1, 2, \dots, n$), the global displacement vector is ordered as

$$
U =
\begin{bmatrix}
u_{x1} \\
u_{y1} \\
\theta_1 \\
u_{x2} \\
u_{y2} \\
\theta_2 \\
\vdots \\
u_{xn} \\
u_{yn} \\
\theta_n
\end{bmatrix}
$$

For node $i$ with 1-based node numbering ($i = 1, 2, \dots, n$),

$$
u_{xi} = U_{3i-2},
\qquad
u_{yi} = U_{3i-1},
\qquad
\theta_i = U_{3i}
$$

If nodes are numbered starting from 0 ($i = 0, 1, \dots, n-1$),

$$
u_{xi} = U_{3i},
\qquad
u_{yi} = U_{3i+1},
\qquad
\theta_i = U_{3i+2}
$$



## Displacement Interpolation in Local Coordinates

The element has nodal degrees of freedom

$$
[\,u_{xi}, u_{yi}, \theta_i, u_{xj}, u_{yj}, \theta_j\,]
$$

### Axial interpolation (local)

The axial displacement is interpolated linearly between the end values

$$
u_x(x) = \left(1-\frac{x}{L}\right)u_{xi} + \frac{x}{L}u_{xj}
$$

### Transverse interpolation (local bending)

The transverse displacement uses the same cubic interpolation as in Problem 1

$$
u_y(x) = N_1(x)u_{yi} + N_2(x)\theta_i + N_3(x)u_{yj} + N_4(x)\theta_j
$$

with

$$
\begin{aligned}
N_1(x) &= 1 - 3\frac{x^2}{L^2} + 2\frac{x^3}{L^3} \\
N_2(x) &= x - 2\frac{x^2}{L} + \frac{x^3}{L^2} \\
N_3(x) &= 3\frac{x^2}{L^2} - 2\frac{x^3}{L^3} \\
N_4(x) &= -\frac{x^2}{L} + \frac{x^3}{L^2}
\end{aligned}
$$


### Important

Do not draw the deformed frame using only straight lines between displaced nodes

Evaluate $u_x(x)$ and $u_y(x)$ at multiple points along each element to obtain a smooth deformed shape

Connected elements must share the same nodal DOFs from the global vector $U$, so joints remain connected and rotations are compatible

### Check Against Your Manual Frame Solution

- Does the frame sway or bend in the correct direction
- Do joints remain connected
- Does the deformed shape look physically reasonable


## Conceptual Steps — Frame Deformation Plot

You can likely reuse much of your code from Problem 1.

1. **Loop over elements**  
   Get node coordinates and compute the element length $L$ and direction cosines $c$, $s$.

2. **Extract global nodal DOFs**  
   From the global vector $U$, extract  
   $[\,u_{xi}, u_{yi}, \theta_i, u_{xj}, u_{yj}, \theta_j\,]$.

3. **Transform global DOFs to local DOFs**  
   Use the element transformation matrix to express these nodal displacements in the local frame.

4. **Interpolate in the local frame**  
   For $x \in [0, L]$, compute axial deformation (linear) and transverse deformation (cubic shape functions).

5. **Transform the local interpolated deformations back to global**  
   Rotate the local displacement field to global coordinates using the rotation matrix.

6. **Plot the result**  
   Add scaled global displacements to the undeformed geometry and draw the deformed curve.

   

In [32]:
def plot_deformed_frame_2d(nodes, elements, U, scale=1.0, npts=50):
    """
    Problem 2 — Frame

    Plot undeformed and deformed shape for a 2D frame

    Inputs
    ------
    nodes : dict
        {node_id: (x, y)}
    elements : dict
        {elem_id: (i, j)}
    U : array-like
        Global displacement vector with [ux, uy, th] per node
    scale : float
        Deformation scale factor
    npts : int
        Number of sample points per element

    Tasks
    -----
    1) Loop through elements
    2) Compute L, c, s
    3) Extract (ux_i, uy_i, th_i, ux_j, uy_j, th_j) from U
    4) Compute ux(x) (linear) and uy(x) (cubic)
    5) Transform local displacement to global
    6) Plot undeformed and deformed structures
    """

    # TODO: implement


In [33]:
# Write any additional helper function(s) that you want to call from your plotting function

In [34]:
# Define nodes, elements, and U for your beam example

nodes = {}
elements = {}
U = []

# Then call plot_deformed_frame_2d(...)

---

## Problem 3 — Complex Frame Deformation

Design and test your plotting function on a **more complex frame structure**.

Move beyond the simple examples from Problems 1 and 2. Create a multi-storey, multi-bay frame with a geometry that is interesting and slightly irregular.

For example, you might consider:

- A 6-storey, 3-bay office frame  
- A stepped or setback building  
- A leaning or asymmetric structure  
- A frame with varying storey heights  

Below is an example of a 10-storey, 1-bay structure subjected to arbitrary displacements:

![Example frame deformation](../Lectures/L7/assets/Assignment_ExampleOutput.png)

Your task is to:

1. Define a new frame geometry (nodes + elements).
2. Create an artificial but nontrivial displacement vector $U$.
3. Plot both the undeformed and deformed structure.

Be creative.

Try assigning:

- Increasing lateral sway with height  
- Alternating rotations  
- Localized distortions  
- Different deformation patterns in different bays  

The goal is to stress-test your plotting function and verify that:

- The deformation remains smooth  
- Joints remain connected  
- Rotations are handled correctly  
- The shape looks physically consistent  

This problem is about geometric reasoning and visualization — not structural correctness.  
Explore how your interpolation and transformation function behaves under complex deformation patterns.

In [35]:
# Define nodes, elements, and U for your beam example

nodes = {}
elements = {}
U = []

# Then call plot_deformed_frame_2d(...)

---

## Reflection (Required)

In 3–6 sentences:

- What part felt easiest? Describe in speciics
- What part felt hardest? Describe in speciics
- One bug you encountered and how you fixed it.
- One thing you still don’t understand.
- If you used collaboration or AI tools, briefly describe how.


### YOUR RESPONSE HERE